In [1]:
import os
import ast
import glob
from scipy.misc import imread, imresize
import numpy as np

data_dir = '../data/dog_vs_cat'

def data_load(data_dir_path, img_height, img_width):
    """

    :param data_dir_path: data home dir
    :return: x, y    x is image content, y is category name
    """
    data_dir = data_dir_path
    subdirs = [os.path.join(data_dir, x) for x in os.listdir(data_dir)]
    x, y = [], []
    for cls in subdirs:
        imgs = glob.glob(os.path.join(cls, '*'))
        tmp_y = os.path.split(cls)[-1]
        for img_path in imgs:
            x.append(imresize(imread(img_path), (img_height, img_width)))
            y.append(tmp_y)
    return np.array(x), y

org_x,org_y = data_load(data_dir,224,224)
print(org_x.shape,org_y[0])

(292, 224, 224, 3) dog


In [2]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape
from keras.layers import Conv2D, MaxPooling2D



# AlexNet with batch normalization in Keras
# input image is 224x224

class AlexNet(object):
    def __init__(self, h, w, num_class):
        self.h = h
        self.w = w
        self.num_class = num_class

    def get_model(self):
        model = Sequential()

        model.add(Conv2D(64, kernel_size=11, input_shape=(self.h, self.w, 3), strides=4, padding='valid'))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(3, 3), strides=2))

        model.add(Conv2D(192, kernel_size=5,padding='valid'))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(384, kernel_size=3, padding='valid'))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(256, kernel_size=3, padding='valid'))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Flatten())
        model.add(Dense(4096))
        model.add(Activation('relu'))
        model.add(Dense(4096))
        model.add(Activation('relu'))
        model.add(Dense(self.num_class, activation='softmax'))

        model.compile(loss='categorical_crossentropy',
                      optimizer='rmsprop',
                      metrics=['accuracy'])

        return model

net = AlexNet(224,224,2)
model = net.get_model()
print(model)

Using TensorFlow backend.


In [3]:
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
cate_y = to_categorical(encoder.fit_transform(org_y))
model.fit(org_x,cate_y,validation_split=0.2,verbose=1,epochs=3)

Train on 233 samples, validate on 59 samples
Epoch 1/3
233/233 [==============================] - 36s - loss: 6.2007 - acc: 0.5365 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 2/3
233/233 [==============================] - 34s - loss: 6.6409 - acc: 0.5880 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 3/3
233/233 [==============================] - 33s - loss: 6.6409 - acc: 0.5880 - val_loss: 16.1181 - val_acc: 0.0000e+00


In [4]:
res = model.predict(org_x[:10])
print(res)
class_type = np.argmax(res[0])
prob = max(res[0])
print(encoder.inverse_transform(class_type), prob, encoder.inverse_transform(0))

[[ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]]
dog 1.0 cat


from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True)  # randomly flip images
model.fit_generator(datagen.flow(org_x,cate_y),steps_per_epoch=32, epochs=10,verbose=1)